In [8]:
from anthropic import Anthropic

In [9]:
import sqlite3

In [25]:
client = Anthropic()
MODEL_NAME = "claude-3-opus-20240229"

In [11]:
#### Creating a Test Database

In [28]:
# connect to the test database ( or create if it doesn't exist)
conn = sqlite3.connect("test_db.db")
cursor = conn.cursor()

# create a sample table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS employees (
               id INTEGER PRIMARY KEY,
               name TEXT,
               department TEXT,
               salary INTEGER
    )
 """)

In [13]:
# Insert sample data
sample_data = [
    (1, "John Doe", "Sales", 50000),
    (2, "Jane Smith", "Engineering", 75000),
    (3, "Mike Johnson", "Sales", 60000),
    (4, "Emily Brown", "Engineering", 80000),
    (5, "David Lee", "Marketing", 55000)
]

In [ ]:
cursor.executemany("INSERT INTO employees VALUES (?, ?, ?, ?)", sample_data)

In [ ]:
conn.commit()

### Generate SQL with Claude

In [22]:
def ask_claude(query, schema):
    prompt = f""" Here is the schema for a database
    {schema}

    Given this schema, can you output a SQL query to answer the following questions? Only output the sql query and nothing else.

    Question:  {query}
    """

    response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2048,
        messages = [
            {'role': 'user', 'content': prompt}
        ]
    )

    return response.content[0].text

In [23]:
### Fetching the schema for the database
schema = cursor.execute("PRAGMA table_info(employees)").fetchall()
schema_str = "CREATE TABLE EMPLOYEES (\n" + "\n".join([f"{col[1]} {col[2]}" for col in schema]) + "\n)"
print(schema_str)

CREATE TABLE EMPLOYEES (
id INTEGER
name TEXT
department TEXT
salary INTEGER
)


In [26]:
# Example natural language question
question = "What are the names and salaries of employees in the Engineering department?"
# Send the question to Claude and get the SQL query
sql_query = ask_claude(question, schema_str)
print(sql_query)

SELECT name, salary
FROM EMPLOYEES
WHERE department = 'Engineering';


In [27]:
# using the above response to check for correctness

results = cursor.execute(sql_query).fetchall()

for result in results:
    print(result)

('Jane Smith', 75000)
('Emily Brown', 80000)


In [29]:
# Example natural language question
question = "Who are thee employees with salary > 10000? Give their name, department and salary"
# Send the question to Claude and get the SQL query
sql_query = ask_claude(question, schema_str)
print(sql_query)

SELECT name, department, salary
FROM EMPLOYEES
WHERE salary > 10000;


In [30]:
# using the above response to check for correctness

results = cursor.execute(sql_query).fetchall()

for result in results:
    print(result)

('John Doe', 'Sales', 50000)
('Jane Smith', 'Engineering', 75000)
('Mike Johnson', 'Sales', 60000)
('Emily Brown', 'Engineering', 80000)
('David Lee', 'Marketing', 55000)


In [31]:
# Example natural language question
question = "Show me the employees whose name start with a vowel or department starts with S"
# Send the question to Claude and get the SQL query
sql_query = ask_claude(question, schema_str)
print(sql_query)

SELECT *
FROM EMPLOYEES
WHERE name LIKE 'A%' OR name LIKE 'E%' OR name LIKE 'I%' OR name LIKE 'O%' OR name LIKE 'U%'
   OR department LIKE 'S%';


In [32]:
# using the above response to check for correctness

results = cursor.execute(sql_query).fetchall()

for result in results:
    print(result)

(1, 'John Doe', 'Sales', 50000)
(3, 'Mike Johnson', 'Sales', 60000)
(4, 'Emily Brown', 'Engineering', 80000)


In [ ]:
conn.close()